In [10]:
pip install geopy requests


In [4]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.5 MB/s eta 0:00:00


In [9]:
pip install gradio

In [19]:
import gradio as gr
import uuid
from geopy.distance import geodesic
import requests
import folium
from threading import Thread
import time

# Coordenadas de la pizzería: Calle Mayor, número 1, Alcalá de Henares
pizzeria_coords = (40.481979, -3.363542)  # Latitud y longitud

# Lista para almacenar pedidos
pedidos = []

# Colores únicos para las motos (7 motos disponibles)
moto_colors = ["blue", "red", "green", "purple", "orange", "darkred", "darkblue"]

# Lista para representar las motos y su estado (libre/ocupada)
motos = [{"id": i, "color": moto_colors[i], "estado": "Libre", "pedidos": []} for i in range(len(moto_colors))]

# Función para generar ID único
def generar_id_pedido():
    return str(uuid.uuid4())[:8]  # Usa los primeros 8 caracteres de un UUID

# Función para obtener coordenadas de una dirección usando la API de Nominatim
def obtener_coordenadas(direccion):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": direccion,
        "format": "json",
        "addressdetails": 1
    }
    headers = {
        "User-Agent": "PizzeriaApp/1.0 (tuemail@dominio.com)"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        return float(data["lat"]), float(data["lon"])
    return None

# Función para obtener rutas óptimas usando OpenRouteService
def obtener_ruta_optima(origen, destino):
    api_key = "5b3ce3597851110001cf6248f0014a6341994d12987eeca0470d048a"
    url = "https://api.openrouteservice.org/v2/directions/driving-car"
    headers = {
        "Authorization": api_key
    }
    params = {
        "start": f"{origen[1]},{origen[0]}",
        "end": f"{destino[1]},{destino[0]}"
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        ruta = response.json()["features"][0]["geometry"]["coordinates"]
        return [(coord[1], coord[0]) for coord in ruta]
    return [origen, destino]

# Función para calcular el tiempo de preparación del pedido
def calcular_tiempo_preparacion(pedido):
    pequenas = pedido["Pequeñas"] * 1  # 1 minuto por pizza pequeña
    medianas = pedido["Medianas"] * 2  # 2 minutos por pizza mediana
    grandes = pedido["Grandes"] * 3  # 3 minutos por pizza grande
    return max(pequenas, medianas, grandes) * 60  # Convertir a segundos

# Función para procesar un pedido
def procesar_pedido(calle, numero, piso, localidad, pizza_pequena, pizza_mediana, pizza_grande):
    localidad_procesada = localidad.strip().lower()

    if localidad_procesada not in ["alcala de henares", "alcalá de henares"]:
        return "Error: Solo repartimos en Alcalá de Henares."

    direccion_simple = f"{calle}, {numero}, {localidad.capitalize()}"
    coords = obtener_coordenadas(direccion_simple)
    if not coords:
        return f"Error: No se pudo determinar la ubicación de la dirección: {direccion_simple}"

    distancia = geodesic(pizzeria_coords, coords).km
    if distancia > 5:
        return f"Error: La dirección está fuera de nuestro radio de entrega (Distancia: {distancia:.2f} km)"

    if pizza_pequena <= 0 and pizza_mediana <= 0 and pizza_grande <= 0:
        return "Por favor, selecciona al menos una pizza antes de finalizar el pedido."

    direccion_completa = f"{calle}, {numero}, Piso {piso}, {localidad.capitalize()}"
    pedido_id = generar_id_pedido()

    pedido = {
        "ID": pedido_id,
        "Dirección": direccion_completa,
        "Coordenadas": coords,
        "Pequeñas": pizza_pequena,
        "Medianas": pizza_mediana,
        "Grandes": pizza_grande,
        "Estado": "En preparación",
        "Moto asignada": None,
        "Ruta": None
    }
    pedidos.append(pedido)

    # Simular tiempo de preparación en un hilo separado
    def preparar_pedido():
        tiempo_preparacion = calcular_tiempo_preparacion(pedido)
        time.sleep(tiempo_preparacion)
        pedido["Estado"] = "Listo para salir"

    Thread(target=preparar_pedido).start()

    return f"Pedido recibido con ID: {pedido_id}\nPizzas - Pequeñas: {pizza_pequena}, Medianas: {pizza_mediana}, Grandes: {pizza_grande}\nDirección: {direccion_completa}\nEstado: En preparación"

# Función para asignar un pedido a una moto
def asignar_pedido_a_moto(pedido_id, moto_id):
    # Buscar el pedido por su ID
    pedido = next((p for p in pedidos if p["ID"] == pedido_id and p["Estado"] == "Listo para salir"), None)
    if not pedido:
        return "Error: Pedido no encontrado o no está listo para salir."

    # Verificar si la moto está disponible y tiene capacidad
    moto = next((m for m in motos if m["id"] == moto_id), None)
    if not moto or moto["estado"] != "Libre" or len(moto["pedidos"]) >= 3:
        return f"Error: La moto {moto_id} no está disponible, no existe, o está llena."

    # Asignar el pedido a la moto
    pedido["Estado"] = "En reparto"
    moto["estado"] = "Ocupada"
    moto["pedidos"].append(pedido_id)
    pedido["Moto asignada"] = moto["id"]

    # Generar la ruta para el pedido
    ruta = obtener_ruta_optima(pizzeria_coords, pedido["Coordenadas"])
    pedido["Ruta"] = ruta

    # Verificar si la moto está llena
    if len(moto["pedidos"]) == 3:
        moto["estado"] = "Ocupada"

    return f"Pedido {pedido_id} asignado a la moto {moto_id}. Ruta generada."

# Función para marcar una moto como libre nuevamente
def liberar_moto(moto_id):
    moto = next((m for m in motos if m["id"] == moto_id), None)
    if not moto:
        return "Error: Moto no encontrada."

    # Marcar los pedidos como entregados y vaciar la lista de la moto
    for pedido_id in moto["pedidos"]:
        pedido = next((p for p in pedidos if p["ID"] == pedido_id), None)
        if pedido:
            pedido["Ruta"] = None
            pedido["Estado"] = "Entregado"
            pedido["Moto asignada"] = None

    moto["pedidos"] = []
    moto["estado"] = "Libre"
    return f"Moto {moto_id} está ahora libre"

# Función para generar el mapa de rutas
def generar_mapa():
    if not pedidos:
        return "No hay pedidos registrados para mostrar en el mapa."

    mapa = folium.Map(location=pizzeria_coords, zoom_start=14)
    folium.Marker(location=pizzeria_coords, tooltip="Pizzería", icon=folium.Icon(color='green')).add_to(mapa)

    for pedido in pedidos:
        if pedido["Estado"] == "En reparto" and pedido["Ruta"]:
            color = moto_colors[pedido["Moto asignada"] % len(moto_colors)]
            folium.Marker(location=pedido["Coordenadas"], tooltip=pedido["Dirección"], icon=folium.Icon(color=color)).add_to(mapa)
            folium.PolyLine(pedido["Ruta"], color=color, weight=2.5, opacity=1).add_to(mapa)

    return mapa._repr_html_()

# Función para mostrar el listado de pedidos
def mostrar_pedidos():
    if not pedidos:
        return "No hay pedidos registrados."

    listado = "Listado de Pedidos:\n"
    for pedido in pedidos:
        listado += f"ID: {pedido['ID']}, Dirección: {pedido['Dirección']}, Estado: {pedido['Estado']}, Moto: {pedido['Moto asignada'] if pedido['Moto asignada'] is not None else 'Sin asignar'}\n"
    return listado

# Interfaz de Gradio
with gr.Blocks() as app:
    gr.Markdown("# Sistema de Pedidos - Pizzería")

    with gr.Tabs():
        with gr.Tab("Realizar Pedido"):
            with gr.Row():
                calle_input = gr.Textbox(label="Introduce tu calle", placeholder="Ejemplo: Calle Mayor")
                numero_input = gr.Textbox(label="Número", placeholder="Ejemplo: 10")
                piso_input = gr.Textbox(label="Piso", placeholder="Ejemplo: 2B")
                localidad_input = gr.Textbox(label="Introduce tu localidad", placeholder="Ejemplo: Alcalá de Henares")
                pizza_pequena_input = gr.Number(label="Cantidad de Pizzas Pequeñas", value=0, precision=0)
                pizza_mediana_input = gr.Number(label="Cantidad de Pizzas Medianas", value=0, precision=0)
                pizza_grande_input = gr.Number(label="Cantidad de Pizzas Grandes", value=0, precision=0)

            finalizar_btn = gr.Button("Finalizar pedido")
            salida = gr.Textbox(label="Estado del Pedido", interactive=False)

            finalizar_btn.click(procesar_pedido, inputs=[calle_input, numero_input, piso_input, localidad_input, pizza_pequena_input, pizza_mediana_input, pizza_grande_input], outputs=salida)

        with gr.Tab("Gestión de Pedidos"):
            pedido_id_input = gr.Textbox(label="ID del Pedido", placeholder="Introduce el ID del pedido")
            moto_id_input = gr.Number(label="ID de la Moto", value=0, precision=0)
            asignar_btn = gr.Button("Asignar pedido a moto")
            liberar_btn = gr.Button("Liberar Moto")

            listado_pedidos = gr.Textbox(label="Listado de Pedidos", interactive=False, lines=10)
            actualizar_btn = gr.Button("Actualizar Listado")

            salida_gestion = gr.Textbox(label="Estado de Gestión", interactive=False)

            asignar_btn.click(asignar_pedido_a_moto, inputs=[pedido_id_input, moto_id_input], outputs=salida_gestion)
            liberar_btn.click(liberar_moto, inputs=[moto_id_input], outputs=salida_gestion)
            actualizar_btn.click(mostrar_pedidos, inputs=[], outputs=listado_pedidos)

        with gr.Tab("Mapa de Entregas"):
            mapa_btn = gr.Button("Mostrar mapa de entregas")
            mapa_html = gr.HTML()

            mapa_btn.click(generar_mapa, inputs=[], outputs=mapa_html)

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d10f203079c7741b17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
